In [3]:
from pdf2image import convert_from_path

pages = convert_from_path(r'docs\patient_details\pd_1.pdf',poppler_path = r'C:\poppler-22.04.0\Library\bin')


In [4]:
pages

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1867x2000 at 0x210520B04C0>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2000x1398 at 0x210520B0310>]

In [5]:
pages[0].show()

In [6]:
import pytesseract

pytesseract.pytesseract.tesseract_cmd=r'C:\Program Files\Tesseract-OCR\tesseract.exe'
text = pytesseract.image_to_string(pages[0],lang='eng')

print(text)

47/12/2020

Patient Medical Record

Patient Information Birth Date
Kathy Crawford May 6 1972
(737) 988-0851 Weight
9264 Ash Dr 95
New York City, 10005 .
United States Height:
190
In Case of Emergency
m _ eee ee
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History

Chicken Pox (Varicella):

IMMUNE IMMUNE
Have you had the Hepatitis B vaccination?

No
List any Medical Problems (asthma, seizures, headaches):

Migraine


In [7]:
import numpy as np
import cv2
from PIL import Image

def preprocess_image(img):
     gray = cv2.cvtColor(np.array(img),cv2.COLOR_BGR2GRAY)
     resized = cv2.resize(gray, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
     processed_image = cv2.adaptiveThreshold(
         resized,
         255,
         cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
         cv2.THRESH_BINARY,
         61,
         11    
     )
     return processed_image

In [8]:
img =  preprocess_image(pages[0])
Image.fromarray(img).show()

text = pytesseract.image_to_string(img,lang='eng')

print(text)

17/12/2020

Patient Medical Record

Patient Information Birth Date

Kathy Crawford May 6 1972

(737) 988-0851 Weight’

9264 Ash Dr 95

New York City, 10005 '

United States Height:
190

In Case of Emergency
ee J
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History
nn ee
Chicken Pox (Varicella): Measies:
IMMUNE

IMMUNE
Have you had the Hepatitis B vaccination?

No

List any Medical Problems (asthma, seizures, headaches}:

Migraine


# <h2>Extract Name</h2>

In [22]:
import re


pattern = 'Patient Information(.*?)\(\d{3}\)'
match = re.findall(pattern,text,flags=re.DOTALL)
match

[' Birth Date\n\nKathy Crawford May 6 1972\n\n']

In [23]:
match[0].strip()

'Birth Date\n\nKathy Crawford May 6 1972'

In [24]:
matches = match[0].replace("Birth Date","").strip()
matches

'Kathy Crawford May 6 1972'

In [31]:
pattern = '((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'

date_matches = re.findall(pattern,matches)

date=date_matches[0][0]

In [32]:
matches.replace(date,'').strip()


'Kathy Crawford'

In [36]:
def remove_noise_from_name(name):
    name = name.replace('Birth Date','').strip()
    date_pattern =  '((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'
    date_matches = re.findall(date_pattern,name)
    
    if date_matches:
        date = date_matches[0][0]
        name = name.replace(date,'').strip()
    
    return name

In [37]:
name = ' Birth Date\n\nKathy Crawford May 6 1972\n\n'

name = remove_noise_from_name(name)
name 

'Kathy Crawford'

# <h2>Extract Phone number</h2>

In [38]:
pattern  = 'Patient Information(.*?)(\(\d{3}\) \d{3}-\d{4})'

matches = re.findall(pattern,text,flags=re.DOTALL)
matches

[(' Birth Date\n\nKathy Crawford May 6 1972\n\n', '(737) 988-0851')]

In [39]:
matches[0][1]

'(737) 988-0851'

<h2>Extract Vaccine info</h2>

In [40]:

pattern = 'Have you had the Hepatitis B vaccination\?.*(Yes|No)'
matches = re.findall(pattern,text,flags=re.DOTALL)
matches

['No']

# <h2>Extract medical problems</h2>

In [46]:
pattern = 'List any Medical Problems \(asthma, seizures, headaches\}:(.*)'

matches = re.findall(pattern,text,flags=re.DOTALL)
matches[0].strip()

'Migraine'